### 4 By 4 Skyscrapers
##### Codewars | 4 kyu | 5671d975d81d6c1c87000022

In a grid of 4x4 squares you want to place a skyscraper in each square with only some clues.

1) The height of the skyscrapers is between 1 and 4 inclusive
2) No two skyscrapers in a row or column may have the same number of floors (i.e. Sodoku Constraint)
3) A clue is the number of skyscrapers that you can see in a row or column from the outside
4) Higher skyscrapers block the view of lower skyscrapers located behind them

Clues wrap each sides of the 4x4 grid from left-to-right and top-to-bottom. We start at the top side, then right side, bottom, and lastly left side. There are 16 total spots for clues. An entry of ```0``` indicates no clue provided. 

**Input:** A list of clues 16 integers in length

**Output:** A 2D Python list detailing the skyscraper heights for the 4x4 grid

**Approach 1:** Rainbow tables for combinations (eliminates unecessary checks and recursive calls) + DFS backtracking

In [42]:
from itertools import permutations
from collections import defaultdict
import numpy as np
import copy

# Opposing pairs of clues
pairs = {
    0: 11, 1: 10, 2: 9, 3: 8,
    4: 15, 5: 14, 6: 13, 7: 12,
    8: 3, 9: 2, 10: 1, 11: 0, 
    12: 7, 13: 6, 14: 5, 15: 4
}

# All permutations of {1, 2, 3, 4}
combo = list(permutations(range(1, 5)))

# Create a dictionary that has the input as the clue (current, opposite)
# and will give a list of indexes of permutations in combos that are 
# potential solutions; 0s are free spaces
table = defaultdict(lambda: [])

for c in combo:
    c0, c1, v0, v1 = 0, 0, 0, 0
    for i in range(4):
        if c[i] > v0:
            c0 += 1
            v0 = c[i]
        if c[3 - i] > v1:
            c1 += 1
            v1 = c[3 - i]
    table[(c0, c1)].append(c)
    table[(c0, 0)].append(c)

def create_valid(l: np.array, check = []) -> np.array:
    '''
    Returns a list of list containing each of the valid
    possible permutations. 0s are treated as free spots.
    If check is not none, the overlap between the output
    of this and check will be returned.
    '''
    v, inds, r, l = [1, 2, 3, 4], [], [], l.copy()

    # Duplicate entries
    if sum(set(l)) != sum(l): return []

    # Processing the input list
    for i, c in enumerate(l): 
        if not c: inds.append(i)
        else: v.remove(c)
    
    # Creating the different list possibilities
    com = list(permutations(v))
    for c in com:
        for i, n in zip(inds, c): 
            l[i] = n
        if len(check) and (tuple(l.tolist()) not in check):
            continue
        r.append(copy.deepcopy(l))
    
    return np.array(r)

def validate_grid(g: np.array) -> bool:
    '''
    Validates a 4x4 grid to avoid preforming uneeded work 
    during DFS traversal. Even though we only use valid entries 
    for a given row or column, invalid combinations can be 
    generated via intra-row/column interactions.
    '''
    for r in g: # Check rows
        if sum(r) != sum(np.unique(r)):
            return False
    
    for c in g.T: # Check columns
        if sum(c) != sum(np.unique(c)):
            return False
    
    return True

def get_coords_values(p: int, grid: np.array) -> tuple:
    '''
    Returns a slice of grid for an input clue position and
    the slicing protocol.
    '''
    match p // 4:
        case 0:
            return grid[:, p], np.s_[:, p]
        case 1:
            return grid[p - 4, :][::-1], np.s_[p - 4, :][::-1]
        case 2:
            return grid[:, 11 - p][::-1], np.s_[:, 11 - p][::-1]
        case 3:
            return grid[15 - p, :], np.s_[15 - p, :]

def solve_puzzle(clues: list[int]) -> list[list[int]]:
    ''' 
    Using smart DFS backtracking, should fill in the 4x4 grid such
    that from each clue, only said skyscrapers can be seen. There are
    no duplicate skyscraper heights for any row or column.
    '''

    def dfs(p: int, g: np.array):
        
        # Base case (validate)
        if not validate_grid(g): return None
        if p == 16: return g

        # Copy to pass down
        g = copy.deepcopy(g)

        # Getting clue values & possible combos
        c0, c1 = clues[p], clues[pairs[p]]
        
        cposs = []
        if c0: # Clue for p
            cposs = table[(c0, c1)]
        elif (not c0) and c1: # No clue for p but clue for opposie side
            cposs = list(map(lambda x: x[::-1], table[(c1, 0)]))
        
        # Valid possibilities
        gvalues, slice = get_coords_values(p, g)
        overlap = create_valid(gvalues, check = cposs)

        # Iterate and call
        for rc in overlap:
            g[slice] = rc
            if list(rc) == [2, 3, 4, 1]:
                print('here')
            r = dfs(p + 1, copy.deepcopy(g))
            if r is not None: return r

    for i in range(16):
        r = dfs(i, np.zeros((4, 4)))
        if r is not None: return r.tolist()

---

### Approach 2 - Brute Force

In [ ]:
import numpy as np
def solve_puzzle(clues: list[int]) -> list[list[int]]:
    
    sol = np.zeros((4, 4), dtype = int)

    def validate_row(c1, c2, r) -> bool:
        v1, v2, c1_, c2_ = 0, 0, c1, c2
        for i in range(4):
            if r[i] > v1:
                c1 -= 1
                v1 = r[i]
            if r[3 - i] > v2:
                c2 -= 1
                v2 = r[3 - i]
        if (c1 == 0 if c1_ > 0 else True) and\
            (c2 == 0 if c2_ > 0 else True):
           return True
        return False

    def check_sol() -> bool:
        for i in range(4):
            if not validate_row(clues[15 - i], clues[4 + i], sol[i, :]) or\
               not validate_row(clues[i], clues[11 - i], sol[:, i]): 
                return False
        return True

    def dfs() -> bool:
        for y in range(4):
            for x in range(4):
                if sol[y][x] == 0:
                    for n in range(1, 5):
                        if (n not in sol[:, x]) and (n not in sol[y, :]):
                            sol[y][x] = n
                            if dfs(): return True
                            sol[y][x] = 0
                    return False
        return check_sol()
                
    dfs()
    return tuple(map(tuple, sol.tolist()))

In [85]:
# 1 3 4 2
# 4 2 1 3
# 3 4 2 1
# 2 1 3 4
solve_puzzle([
    2, 2, 1, 3,
    2, 2, 3, 1,
    1, 2, 2, 3,
    3, 2, 1, 3
])

((1, 3, 4, 2), (4, 2, 1, 3), (3, 4, 2, 1), (2, 1, 3, 4))

In [83]:
# 2 1 4 3
# 3 4 1 2
# 4 2 3 1
# 1 3 2 4
solve_puzzle([
    0, 0, 1, 2,
    0, 2, 0, 0,
    0, 3, 0, 0,
    0, 1, 0, 0
])

[[2.0, 1.0, 4.0, 3.0],
 [3.0, 4.0, 1.0, 2.0],
 [4.0, 2.0, 3.0, 1.0],
 [1.0, 3.0, 2.0, 4.0]]